In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# This nodebook extracts poses from a data source

from pretrained import FaceDetector

In [ ]:
detector = FaceDetector()

In [ ]:
from PIL import Image
from IPython.display import display
from pathlib import Path
from tqdm import tqdm

from constants import emotions

processed_dir = Path("/media/regieki/data/RegieKI/data/processed4")
device_paths = [f for f in processed_dir.iterdir() if f.is_dir()]



In [ ]:

# Make face montages

data_dir = Path("/media/regieki/data/RegieKI/data/all_faces2") 

for emotion in emotions:
    i = 0
    for device_path in device_paths:
        device = device_path.stem
        print(f"Starting {device} -> {emotion}")
        image_dir  = Path(device_path, emotion)

        for image_path in tqdm(image_dir.glob("*.png")):
            output_dir = Path(data_dir, device, emotion)

            print(output_dir.resolve())
            
            if(not output_dir.exists()):
                output_dir.mkdir(parents=True, exist_ok=True)

            face_path  = Path(output_dir, f"image_{i:06d}.jpg")

            if( face_path.exists()):
                continue

            img = Image.open(image_path).convert("RGB")
            faces = detector.get_faces(img)

            for face in faces:                
                i += 1
                face.save(face_path)

In [ ]:

import random
import subprocess

data_dir = Path("/media/regieki/data/RegieKI/data/all_faces2") 
video_dir = Path("/media/regieki/data/RegieKI/data/videos") 

for leaf_dir in data_dir.glob("*/*/"):

    i = 0
    files = list( leaf_dir.glob("image_*.jpg"))
    random.shuffle(files)
    for file in files:
        
        newName = Path(leaf_dir, f"shuffled_image_{i:06d}.jpeg")
        print(newName.resolve())
        file.rename(newName)
        i += 1


    print(leaf_dir)
    device  = leaf_dir.parts[-2]
    emotion = leaf_dir.parts[-1]

    filename = f"video_{device}_{emotion}.mp4"

    video_path = Path(video_dir, filename)
    image_path = f"{leaf_dir.resolve()}/shuffled_image_%06d.jpeg"

    cmd = f"ffmpeg -y -r 60 -i {image_path} -c:v libx264 -vf 'format=yuv420p, pad=ceil(iw/2)*2:ceil(ih/2)*2'  {video_path.resolve()}"
    print(cmd)
    try:
        subprocess.run([cmd], capture_output=True,  shell=True, check=True)
    except:
        print("Failed")
